# block_NN

In [2]:
import tensorflow as tf
import numpy as np
from pprint import pprint
import sys, os

In [33]:
class Block_NN(object):
    
    
    def __init__(self,
                 name, 
                 num_downsamples,
                 log_dir = 'logs/',
                 dtype = tf.float32,
                 adam_inital_learning_rate = 0.0001,
                 adam_epsilon = 1e-8,
                 verbose = False):
    
        self.name = name
        self.num_downsamples = num_downsamples
        self.log_dir = log_dir
        self.dtype = dtype
        self.adam_inital_learning_rate = adam_inital_learning_rate
        self.adam_epsilon = adam_epsilon
        
        self._verbose = verbose
        self._num_freq_channels = 1024
        self._layers = []
        self._msg = ''
        self._num_classes = 161
        
        self._vprint = sys.stdout.write if self._verbose else lambda *a, **k: None
        
    def print_params(self):
        """Prints netwwork parameters"""
        pprint(self._gen_params_dict())
        
    def _gen_params_dict(self):
        """Generates a dictionary of the arguments, the parameters that make this network."""
        d = self.__dict__
        return {key : d[key] for key in d.keys() if key[0] != '_' if 'tensorflow' not in str(type(d[key]))}
        
    def _save_params(self):
        """Saves the parameters of the network as a dictionary in a .npz file .
           path = log_dir/name/params/Splits.npz
         """
        self._msg += '\rsaving network parameters'
        self._vprint(self._msg)
        direc = self.log_dir + self.name + '/params/'

        if not os.path.exists(direc):
            self._msg += '- creating new directory'
            self._vprint(self._msg)
            os.makedirs(direc)
            
        np.savez(direc + self.__class__.__name__, self._gen_params_dict())
        self._msg += ' - params saved'
        self._vprint(self._msg)
        
    def load_params(self, path):
        """Load in the parameters of an old network, but keep the current network name."""
    
        self._msg = '\rloading network parameters'
        self._vprint(self._msg)
        
        a = np.load(path + '.npz')
        d = dict(zip(("data1{}".format(k) for k in a), (a[k] for k in a)))
        
        self._msg += ' - setting params'
        self._vprint(self._msg)
        
        name = self.name
        params = d['data1arr_0'][()]
        for key in params:
            setattr(self, key, params[key])
        self.name = name
        
        self._msg = '\rparams loaded'
        self._vprint(self._msg)
        
    def create_graph(self):

        self._save_params()
        self._msg = '\rcreating network graph '; self._vprint(self._msg)

        
        tf.reset_default_graph()
        self.is_training = tf.placeholder(dtype = tf.bool, shape = [], name = 'is_training')
        
        with tf.variable_scope('keep_probs'):
            
            self._msg += '.'; self._vprint(self._msg)

            self.sample_keep_prob = tf.placeholder(self.dtype, name = 'sample_keep_prob')
            self.downsample_keep_prob = tf.placeholder(self.dtype, name = 'downsample_keep_prob')
        
        
        with tf.variable_scope('sample'):
            
            self._msg += '.'; self._vprint(self._msg)
            
            self.X = tf.placeholder(self.dtype, shape = [None, 1, self._num_freq_channels, 1], name = 'X')
            self.X = tf.nn.dropout(self.X, self.sample_keep_prob)
            
        trainable = lambda shape, name : tf.get_variable(name = name,
                                                         dtype = self.dtype,
                                                         shape = shape,
                                                         initializer = tf.contrib.layers.xavier_initializer())
        for i in range(self.num_downsamples):
            self._msg += ' {}'.format(i); self._vprint(self._msg)
            
            with tf.variable_scope('conv_layer_{}'.format(i)):

                layer = self.X if i == 0 else self._layers[-1]
                
                # filter shape:
                fh = 1 # filter height = 1 for 1D convolution
                fw = np.max([layer.get_shape().as_list()[2] / 4, 2])# filter width
                fic = 4**(i) # num in channels = number of incoming filters
                foc = 4**(i+1) # num out channels = number of outgoing filters
                filters = trainable([fh, fw, fic, foc], 'filters')
                
                # stride shape
                sN = 1 # batch = 1 (why anything but 1 ?)
                sH = 1 # height of stride = 1 for 1D conv
                sW = 1 # width of stride = downsampling factor = 1 for no downsampling or > 1 for downsampling
                sC = 1 # depth = number of channels the stride walks over = 1 (why anything but 1 ?)
                strides_no_downsample = [sN, sH, sW, sC]
                layer = tf.nn.conv2d(layer, filters, strides_no_downsample, 'SAME')
                
                # shape of biases = [num outgoing filters]
                biases = trainable([foc], 'biases')
                layer = tf.nn.bias_add(layer, biases)
                layer = tf.nn.leaky_relu(layer)
                layer = tf.contrib.layers.batch_norm(layer, is_training = self.is_training)
                
                # downsample
                with tf.variable_scope('downsample'):
                    fw = fw*2
                    filters = trainable([fh, fw, foc, foc], 'filters')

                    sW = 2
                    strides_no_downsample = [sN, sH, sW, sC]
                    layer = tf.nn.conv2d(layer, filters, strides_no_downsample, 'SAME')

                    # shape of biases = [num outgoing filters]
                    biases = trainable([foc], 'biases')
                    layer = tf.nn.bias_add(layer, biases)
                    layer = tf.nn.leaky_relu(layer)
                    layer = tf.nn.dropout(layer, self.downsample_keep_prob)
                    layer = tf.contrib.layers.batch_norm(layer, is_training = self.is_training)

                self._layers.append(layer)
                    

#         self.layer_nodes = 2**np.arange(0,10)[:self.num_downsamples][::-1]
#         for j in range(len(self.layer_nodes)):
            
#             self._msg += ' {}'.format(i + 1 + j);self._vprint(self._msg)
            
#             with tf.variable_scope('fc_layer_{}'.format(j)):
#                 layer = tf.contrib.layers.fully_connected(self._layers[-1], self.layer_nodes[j])
#                 layer = tf.contrib.layers.batch_norm(layer, is_training = self.is_training)
#                 self._layers.append(layer)
            
            
#         self._msg += ' {}'.format(i + 1 + j);self._vprint(self._msg)

#         with tf.variable_scope('final_layer'):
#             layer = tf.contrib.layers.fully_connected(tf.layers.flatten(self._layers[-1]), self._num_classes)
#             layer = tf.contrib.layers.batch_norm(layer, is_training = self.is_training)
#             self._layers.append(layer)                
        
                
        self._msg += ' '
        with tf.variable_scope('labels'):
            self._msg += '.'; self._vprint(self._msg)
            
            self.labels = tf.placeholder(dtype = self.dtype, shape = [None, self._num_classes], name = 'labels')
            self.true_cls = tf.argmax(self.labels, axis = 1)
            
        with tf.variable_scope('logits'):
            self._msg += '.'; self._vprint(self._msg)
            
            self._logits = tf.contrib.layers.fully_connected(tf.layers.flatten(self._layers[-1]), self._num_classes, activation_fn = None)

        with tf.variable_scope('predictions'):
            self._msg += '.'; self._vprint(self._msg)
            
            self.predictions = tf.nn.softmax(self._logits)
            
            self.pred_cls = tf.argmax(self.predictions, axis = 1)

            self.correct_prediction = tf.equal(self.pred_cls, self.true_cls)
            
            self.accuracy = tf.reduce_mean(tf.cast(self.correct_prediction, self.dtype))
        
        
        with tf.variable_scope('costs'):
            self._msg += '.'; self._vprint(self._msg)

            self._cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(labels = self.labels, logits = self._logits)
 
            self.cost = tf.reduce_mean(self._cross_entropy)
        
        update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
        with tf.control_dependencies(update_ops):
            self._msg += '.'; self._vprint(self._msg)
            self.optimizer = tf.train.AdamOptimizer(self.adam_inital_learning_rate, epsilon=self.adam_epsilon).minimize(self.cost)
            
        with tf.variable_scope('logging'):
            self._msg += '.'; self._vprint(self._msg)
            
            tf.summary.scalar(name = 'cost', tensor = self.cost)
            tf.summary.scalar(name = 'accuracy', tensor = self.accuracy)

            self.summary = tf.summary.merge_all()
            
        self._msg += ' done'
        self._vprint(self._msg)
        self._msg = ''

In [23]:
class Block_NN_2(object):
    
    
    def __init__(self,
                 name, 
                 num_downsamples,
                 log_dir = 'logs/',
                 dtype = tf.float32,
                 adam_inital_learning_rate = 0.0001,
                 adam_epsilon = 1e-8,
                 verbose = False):
    
        self.name = name
        self.num_downsamples = num_downsamples
        self.log_dir = log_dir
        self.dtype = dtype
        self.adam_inital_learning_rate = adam_inital_learning_rate
        self.adam_epsilon = adam_epsilon
        
        self._verbose = verbose
        self._num_freq_channels = 1024
        self._layers = []
        self._msg = ''
        self._num_classes = 161
        
        self._vprint = sys.stdout.write if self._verbose else lambda *a, **k: None
        
    def print_params(self):
        """Prints netwwork parameters"""
        pprint(self._gen_params_dict())
        
    def _gen_params_dict(self):
        """Generates a dictionary of the arguments, the parameters that make this network."""
        d = self.__dict__
        return {key : d[key] for key in d.keys() if key[0] != '_' if 'tensorflow' not in str(type(d[key]))}
        
    def _save_params(self):
        """Saves the parameters of the network as a dictionary in a .npz file .
           path = log_dir/name/params/Splits.npz
         """
        self._msg += '\rsaving network parameters'
        self._vprint(self._msg)
        direc = self.log_dir + self.name + '/params/'

        if not os.path.exists(direc):
            self._msg += '- creating new directory'
            self._vprint(self._msg)
            os.makedirs(direc)
            
        np.savez(direc + self.__class__.__name__, self._gen_params_dict())
        self._msg += ' - params saved'
        self._vprint(self._msg)
        
    def load_params(self, path):
        """Load in the parameters of an old network, but keep the current network name."""
    
        self._msg = '\rloading network parameters'
        self._vprint(self._msg)
        
        a = np.load(path + '.npz')
        d = dict(zip(("data1{}".format(k) for k in a), (a[k] for k in a)))
        
        self._msg += ' - setting params'
        self._vprint(self._msg)
        
        name = self.name
        params = d['data1arr_0'][()]
        for key in params:
            setattr(self, key, params[key])
        self.name = name
        
        self._msg = '\rparams loaded'
        self._vprint(self._msg)
        
    def create_graph(self):

        self._save_params()
        self._msg = '\rcreating network graph '; self._vprint(self._msg)

        
        tf.reset_default_graph()
        self.is_training = tf.placeholder(dtype = tf.bool, shape = [], name = 'is_training')
        
        with tf.variable_scope('keep_probs'):
            
            self._msg += '.'; self._vprint(self._msg)

            self.sample_keep_prob = tf.placeholder(self.dtype, name = 'sample_keep_prob')
            self.downsample_keep_prob = tf.placeholder(self.dtype, name = 'downsample_keep_prob')
        
        
        with tf.variable_scope('sample'):
            
            self._msg += '.'; self._vprint(self._msg)
            
            self.X = tf.placeholder(self.dtype, shape = [None, 1, self._num_freq_channels, 1], name = 'X')
            self.X = tf.nn.dropout(self.X, self.sample_keep_prob)
            
        trainable = lambda shape, name : tf.get_variable(name = name,
                                                         dtype = self.dtype,
                                                         shape = shape,
                                                         initializer = tf.contrib.layers.xavier_initializer())
#         for i in range(self.num_downsamples):
#             self._msg += ' {}'.format(i); self._vprint(self._msg)
            
#             with tf.variable_scope('conv_layer_{}'.format(i)):

#                 layer = self.X if i == 0 else self._layers[-1]
                
#                 # filter shape:
#                 fh = 1 # filter height = 1 for 1D convolution
#                 fw = 3 # filter width
#                 fic = 2**(i) # num in channels = number of incoming filters
#                 foc = 2**(i+1) # num out channels = number of outgoing filters
#                 filters = trainable([fh, fw, fic, foc], 'filters')
                
#                 # stride shape
#                 sN = 1 # batch = 1 (why anything but 1 ?)
#                 sH = 1 # height of stride = 1 for 1D conv
#                 sW = 1 # width of stride = downsampling factor = 1 for no downsampling or > 1 for downsampling
#                 sC = 1 # depth = number of channels the stride walks over = 1 (why anything but 1 ?)
#                 strides_no_downsample = [sN, sH, sW, sC]
#                 layer = tf.nn.conv2d(layer, filters, strides_no_downsample, 'SAME')
                
#                 # shape of biases = [num outgoing filters]
#                 biases = trainable([foc], 'biases')
#                 layer = tf.nn.bias_add(layer, biases)
#                 layer = tf.nn.leaky_relu(layer)
#                 layer = tf.contrib.layers.batch_norm(layer, is_training = self.is_training)
                
#                 # downsample
#                 with tf.variable_scope('downsample'):
#                     fw = 7
#                     filters = trainable([fh, fw, foc, foc], 'filters')

#                     sW = 2
#                     strides_no_downsample = [sN, sH, sW, sC]
#                     layer = tf.nn.conv2d(layer, filters, strides_no_downsample, 'SAME')

#                     # shape of biases = [num outgoing filters]
#                     biases = trainable([foc], 'biases')
#                     layer = tf.nn.bias_add(layer, biases)
#                     layer = tf.nn.leaky_relu(layer)
#                     layer = tf.nn.dropout(layer, self.downsample_keep_prob)
#                     layer = tf.contrib.layers.batch_norm(layer, is_training = self.is_training)

#                 self._layers.append(layer)
                    

        self.layer_nodes = 2**np.arange(0,10)[:self.num_downsamples][::-1]
        for j in range(len(self.layer_nodes)):
            
            self._msg += ' {}'.format(i + 1 + j);self._vprint(self._msg)
            
            with tf.variable_scope('fc_layer_{}'.format(j)):
                input = self.X if j == 0 else self._layers[-1]
                layer = tf.contrib.layers.fully_connected(tf.layers.flatten(input), self.layer_nodes[j])
                layer = tf.contrib.layers.batch_norm(layer, is_training = self.is_training)
                self._layers.append(layer)
            
            
#         self._msg += ' {}'.format(i + 1 + j);self._vprint(self._msg)

#         with tf.variable_scope('final_layer'):
#             layer = tf.contrib.layers.fully_connected(tf.layers.flatten(self._layers[-1]), self._num_classes)
#             layer = tf.contrib.layers.batch_norm(layer, is_training = self.is_training)
#             self._layers.append(layer)                
        
                
        self._msg += ' '
        with tf.variable_scope('labels'):
            self._msg += '.'; self._vprint(self._msg)
            
            self.labels = tf.placeholder(dtype = self.dtype, shape = [None, self._num_classes], name = 'labels')
            self.true_cls = tf.argmax(self.labels, axis = 1)
            
        with tf.variable_scope('logits'):
            self._msg += '.'; self._vprint(self._msg)
            
            self._logits = tf.contrib.layers.fully_connected(tf.layers.flatten(self._layers[-1]), self._num_classes, activation_fn = None)

        with tf.variable_scope('predictions'):
            self._msg += '.'; self._vprint(self._msg)
            
            self.predictions = tf.nn.softmax(self._logits)
            
            self.pred_cls = tf.argmax(self.predictions, axis = 1)

            self.correct_prediction = tf.equal(self.pred_cls, self.true_cls)
            
            self.accuracy = tf.reduce_mean(tf.cast(self.correct_prediction, self.dtype))
        
        
        with tf.variable_scope('costs'):
            self._msg += '.'; self._vprint(self._msg)

            self._cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(labels = self.labels, logits = self._logits)
 
            self.cost = tf.reduce_mean(self._cross_entropy)
        
        update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
        with tf.control_dependencies(update_ops):
            self._msg += '.'; self._vprint(self._msg)
            self.optimizer = tf.train.AdamOptimizer(self.adam_inital_learning_rate, epsilon=self.adam_epsilon).minimize(self.cost)
            
        with tf.variable_scope('logging'):
            self._msg += '.'; self._vprint(self._msg)
            
            tf.summary.scalar(name = 'cost', tensor = self.cost)
            tf.summary.scalar(name = 'accuracy', tensor = self.accuracy)

            self.summary = tf.summary.merge_all()
            
        self._msg += ' done'
        self._vprint(self._msg)
        self._msg = ''

In [24]:
x = Block_NN_2('x',10)
x.create_graph()
x._layers

[<tf.Tensor 'fc_layer_0/BatchNorm/Reshape_1:0' shape=(?, 512) dtype=float32>,
 <tf.Tensor 'fc_layer_1/BatchNorm/Reshape_1:0' shape=(?, 256) dtype=float32>,
 <tf.Tensor 'fc_layer_2/BatchNorm/Reshape_1:0' shape=(?, 128) dtype=float32>,
 <tf.Tensor 'fc_layer_3/BatchNorm/Reshape_1:0' shape=(?, 64) dtype=float32>,
 <tf.Tensor 'fc_layer_4/BatchNorm/Reshape_1:0' shape=(?, 32) dtype=float32>,
 <tf.Tensor 'fc_layer_5/BatchNorm/Reshape_1:0' shape=(?, 16) dtype=float32>,
 <tf.Tensor 'fc_layer_6/BatchNorm/Reshape_1:0' shape=(?, 8) dtype=float32>,
 <tf.Tensor 'fc_layer_7/BatchNorm/Reshape_1:0' shape=(?, 4) dtype=float32>,
 <tf.Tensor 'fc_layer_8/BatchNorm/Reshape_1:0' shape=(?, 2) dtype=float32>,
 <tf.Tensor 'fc_layer_9/BatchNorm/Reshape_1:0' shape=(?, 1) dtype=float32>]

In [4]:
import sys, os

sys.path.insert(1, os.path.join(sys.path[0], '../modules'))
import notebook_loading

from block_Data import *

importing Jupyter notebook from block_Data.ipynb


In [5]:
# extract the redundant baselines and their gains and data from miriad and calfits files
red_bls, gains, uvd = load_relevant_data('../zen_data/zen.2458098.58037.xx.HH.uv','../zen_data/zen.2458098.58037.xx.HH.uv.abs.calfits')

# seperate trining and testing redundant baselines 
# if we have not already done this, load them from disk
training_redundant_baselines_dict, testing_redundant_baselines_dict = get_or_gen_test_train_red_bls_dicts(red_bls, gains.keys())

# seperate the visiblites
training_baselines_data = get_seps_data(training_redundant_baselines_dict, uvd)
testing_baselines_data = get_seps_data(testing_redundant_baselines_dict, uvd)

In [35]:
import matplotlib.pyplot as plt
import io

class Block_Trainer(object):
    
    def __init__(self,
                 network = None,
                 abs_min_max_delay = 0.040,
                 num_flatnesses = 100,
                 num_epochs = 100,
                 batch_size = 32,
                 log_dir = 'logs/',
                 model_save_interval = 25,
                 pretrained_model_path = None,
                 sample_keep_prob = 0.80,
                 downsample_keep_prob = 0.50,
                 verbose = False):
        
        self.network = network
        self.abs_min_max_delay = abs_min_max_delay
        self.num_flatnesses = num_flatnesses
        self.num_epochs = num_epochs
        self.batch_size = batch_size
        self.log_dir = log_dir
        self.model_save_interval = model_save_interval
        self.pretrained_model_path = pretrained_model_path
        self.sample_keep_prob = sample_keep_prob
        self.downsample_keep_prob = downsample_keep_prob
        
        self._num_classes = 161
        
        self._verbose = verbose
        self._msg = ''
        self._vprint = sys.stdout.write if self._verbose else lambda *a, **k: None
        
    def print_params(self):
        """Prints netwwork parameters"""
        pprint(self._gen_params_dict())
        
    def _gen_params_dict(self):
        """Generates a dictionary of the arguments, the parameters that make this network."""
        d = self.__dict__
        return {key : d[key] for key in d.keys() if key != 'network' if key[0] != '_' if 'tensorflow' not in str(type(d[key]))}
        
    def _save_params(self):
        """Saves the parameters of the trainer as a dictionary in a .npz file .
           path = log_dir/name/params/Train.npz
         """
        self._msg += '\rsaving trainer parameters'
        self._vprint(self._msg)
        direc = self.log_dir + self.network.name + '/params/'

        if not os.path.exists(direc):
            self._msg += ' - creating new directory'
            self._vprint(self._msg)
            os.makedirs(direc)
            
        np.savez(direc + self.__class__.__name__, self._gen_params_dict())
        self._msg += ' - params saved'
        self._vprint(self._msg)
        
    def load_params(self, path):
        """Load in the parameters of an old trainer."""
    
        self._msg = '\rloading trainer parameters'
        self._vprint(self._msg)
        
        a = np.load(path + '.npz')
        d = dict(zip(("data1{}".format(k) for k in a), (a[k] for k in a)))
        
        self._msg += ' - setting params'
        self._vprint(self._msg)
        
        params = d['data1arr_0'][()]
        for key in params:
            setattr(self, key, params[key])
        
        self._msg = '\rparams loaded'
        self._vprint(self._msg)



    def add_data(self,train_info, test_info, gains):
        self._msg = '\radding data';self._vprint(self._msg)

        
        self._train_batcher = block_Data_Creator(self.num_flatnesses,
                                           train_info[0],
                                           train_info[1],
                                           gains,
                                           self.abs_min_max_delay)
        self._train_batcher.gen_data()


        self._test_batcher = block_Data_Creator(self.num_flatnesses,
                                          test_info[0],
                                          test_info[1],
                                          gains,
                                          self.abs_min_max_delay)
        
        self._test_batcher.gen_data()
        self._msg += ' - trainer ready';self._vprint(self._msg)
        

        
    def train(self):
        
        self._save_params()
        self.costs = []
        self.accuracies = []

        tf.reset_default_graph()
        
        self.network.create_graph()
        print(np.sum([np.prod(v.get_shape().as_list()) for v in tf.trainable_variables()]))
        saver = tf.train.Saver()
        
        with tf.Session() as session:
            
            if self.pretrained_model_path == None:
                session.run(tf.global_variables_initializer())
                
            else:
                saver.restore(session, self.pretrained_model_path)
                
            
            archive_loc = self.log_dir + self.network.name
            training_writer = tf.summary.FileWriter(archive_loc + '/training', session.graph)
            testing_writer = tf.summary.FileWriter(archive_loc + '/testing', session.graph)
            self.model_save_location = archive_loc + '/trained_model.ckpt'   
            
            
            self._msg = '\rtraining';self._vprint(self._msg)

            try:
                for epoch in range(self.num_epochs):
                    
                    

                    training_inputs, training_labels = self._train_batcher.get_data(); self._train_batcher.gen_data()
                    testing_inputs, testing_labels = self._test_batcher.get_data(); self._test_batcher.gen_data()  

                    training_labels = np.asarray(training_labels)
                    testing_labels = np.asarray(testing_labels).reshape(-1,self._num_classes)
                    

                    
                    # if the division here has a remainde some values are just truncated
                    batch_size = self.batch_size
                    num_entries = self.num_flatnesses * 60.

                    for j in range(int(num_entries/batch_size)):
                        self._msg = '\rEpoch - {:5.0f}/{:5.0f} - batch: {:4.0f}/{:4.0f}'.format(epoch + 1,self.num_epochs, j + 1, int(num_entries/batch_size))
                        if epoch != 0:
                            self._msg += ' - (Training, Testing) - '.format(epoch)
                            self._msg += " costs: ({:0.4f}, {:0.4f})".format(training_cost, testing_cost)
                            self._msg += " accss: ({:0.4f}, {:0.4f})".format(training_acc, testing_acc)
                        self._vprint(self._msg); 

                        training_inputs_batch = training_inputs[j*batch_size:(j + 1)*batch_size].reshape(-1,1,1024,1)
                        training_labels_batch = training_labels[j*batch_size:(j + 1)*batch_size].reshape(-1,self._num_classes)

                        feed_dict = {self.network.X: training_inputs_batch,
                                     self.network.labels: training_labels_batch,
                                     self.network.sample_keep_prob : self.sample_keep_prob,
                                     self.network.downsample_keep_prob : self.downsample_keep_prob,
                                     self.network.is_training : True}

                        session.run([self.network.optimizer], feed_dict = feed_dict) 
                            
                    train_feed_dict = {self.network.X: training_inputs.reshape(-1,1,1024,1),
                                      self.network.labels: training_labels.reshape(-1,self._num_classes),
                                      self.network.sample_keep_prob : 1.,
                                      self.network.downsample_keep_prob : 1.,
                                      self.network.is_training : False}


                    training_cost, training_acc, training_summary = session.run([self.network.cost,
                                                                                 self.network.accuracy,
                                                                                 self.network.summary],
                                                                                 feed_dict = train_feed_dict) 

                    training_writer.add_summary(training_summary, epoch)
                    training_writer.flush()  
                
                    
                    test_feed_dict = {self.network.X: testing_inputs.reshape(-1,1,1024,1),
                                      self.network.labels: testing_labels.reshape(-1,self._num_classes),
                                      self.network.sample_keep_prob : 1.,
                                      self.network.downsample_keep_prob : 1.,
                                      self.network.is_training : False} 

                    testing_cost, testing_acc, testing_summary = session.run([self.network.cost,
                                                                              self.network.accuracy,
                                                                              self.network.summary],
                                                                              feed_dict = test_feed_dict)

                    testing_writer.add_summary(testing_summary, epoch)
                    testing_writer.flush()
                    
                    self.costs.append((training_cost, testing_cost))
                    self.accuracies.append((training_acc, testing_acc))
                    
                    if (epoch + 1) % self.model_save_interval == 0:
                        saver.save(session, self.model_save_location, epoch + 1)
                
                self.msg = ''
            except KeyboardInterrupt:
                self._msg = ' TRAINING INTERUPPTED' # this never prints I dont know why
                pass

            self._msg += '\rtraining ended'; self._vprint(self._msg)
            
            training_writer.close()
            testing_writer.close()


        session.close()
        self._msg += ' - session closed'; self._vprint(self._msg)
        self._msg = ''
        
    def plot(self, mode = 'all', figsize = (8,6) ):
        
        xvals = np.arange(len(self.costs))
        
        
        if mode == 'all':
            
            yvals = [self.costs, self.accuracies]
            metric_names = ['costs', 'accuracies']
            
        fig, axes = plt.subplots(len(yvals), 1, figsize = figsize, dpi = 144)
        
        for i, ax in enumerate(axes.reshape(-1)):
            
            if i == 0:
                ax.set_title('{}'.format(self.network.name))

            ax.plot(xvals, yvals[i], lw = 0.5)
            ax.set_xlabel('Epoch')
            ax.set_ylabel(metric_names[i])
            
        
        plt.tight_layout()
        plt.show()
        
    def save_results(self):
        
        results = {'MISGs': self.MISGs, 'MSEs': self.MSEs, 'PWTs': self.PWTs}
        direc = self.log_dir + self.network.name + '/results/'
        """Not safe - will overwrite existing file."""
        if not os.path.exists(direc):
            os.makedirs(direc)
        np.savez(direc + 'results', results)

In [37]:
num_downsamples = 10
batch_size = 100
lrp = 4
lr = 1.*10.**(-lrp)
network = Block_NN('network_{}_{}_{}'.format(num_downsamples,batch_size, lrp),num_downsamples, adam_inital_learning_rate=lr)
trainer = Block_Trainer(network = network, num_flatnesses = 250, downsample_keep_prob = 0.8, batch_size=batch_size, verbose=True, num_epochs = 5000)
trainer.add_data((training_baselines_data, training_redundant_baselines_dict),
         (testing_baselines_data, testing_redundant_baselines_dict),
         gains)
trainer.train()

training ended - session closed

In [7]:
num_downsamples = 2
batch_size = 2
lrp = 4
lr = 1.*10.**(-lrp)
network = Block_NN('network_{}_{}_{}'.format(num_downsamples,batch_size, lrp),num_downsamples, adam_inital_learning_rate=lr)
trainer = Block_Trainer(network = network, num_flatnesses = 1, downsample_keep_prob = 0.8, batch_size=batch_size, verbose=True, num_epochs = 5000)
trainer.add_data((training_baselines_data, training_redundant_baselines_dict),
         (testing_baselines_data, testing_redundant_baselines_dict),
         gains)
trainer.train()

training ended - session closed

In [8]:
num_downsamples = 3
batch_size = 2
lrp = 4
lr = 1.*10.**(-lrp)
network = Block_NN('network_{}_{}_{}'.format(num_downsamples,batch_size, lrp),num_downsamples, adam_inital_learning_rate=lr)
trainer = Block_Trainer(network = network, num_flatnesses = 1, downsample_keep_prob = 0.8, batch_size=batch_size, verbose=True, num_epochs = 5000)
trainer.add_data((training_baselines_data, training_redundant_baselines_dict),
         (testing_baselines_data, testing_redundant_baselines_dict),
         gains)
trainer.train()

training ended - session closed

In [9]:
num_downsamples = 4
batch_size = 2
lrp = 4
lr = 1.*10.**(-lrp)
network = Block_NN('network_{}_{}_{}'.format(num_downsamples,batch_size, lrp),num_downsamples, adam_inital_learning_rate=lr)
trainer = Block_Trainer(network = network, num_flatnesses = 1, downsample_keep_prob = 0.8, batch_size=batch_size, verbose=True, num_epochs = 5000)
trainer.add_data((training_baselines_data, training_redundant_baselines_dict),
         (testing_baselines_data, testing_redundant_baselines_dict),
         gains)
trainer.train()

training ended - session closed

In [10]:
num_downsamples = 5
batch_size = 2
lrp = 4
lr = 1.*10.**(-lrp)
network = Block_NN('network_{}_{}_{}'.format(num_downsamples,batch_size, lrp),num_downsamples, adam_inital_learning_rate=lr)
trainer = Block_Trainer(network = network, num_flatnesses = 1, downsample_keep_prob = 0.8, batch_size=batch_size, verbose=True, num_epochs = 5000)
trainer.add_data((training_baselines_data, training_redundant_baselines_dict),
         (testing_baselines_data, testing_redundant_baselines_dict),
         gains)
trainer.train()

training ended - session closed

In [11]:
num_downsamples = 6
batch_size = 2
lrp = 4
lr = 1.*10.**(-lrp)
network = Block_NN('network_{}_{}_{}'.format(num_downsamples,batch_size, lrp),num_downsamples, adam_inital_learning_rate=lr)
trainer = Block_Trainer(network = network, num_flatnesses = 1, downsample_keep_prob = 0.8, batch_size=batch_size, verbose=True, num_epochs = 5000)
trainer.add_data((training_baselines_data, training_redundant_baselines_dict),
         (testing_baselines_data, testing_redundant_baselines_dict),
         gains)
trainer.train()

training ended - session closed

In [12]:
num_downsamples = 7
batch_size = 2
lrp = 4
lr = 1.*10.**(-lrp)
network = Block_NN('network_{}_{}_{}'.format(num_downsamples,batch_size, lrp),num_downsamples, adam_inital_learning_rate=lr)
trainer = Block_Trainer(network = network, num_flatnesses = 1, downsample_keep_prob = 0.8, batch_size=batch_size, verbose=True, num_epochs = 5000)
trainer.add_data((training_baselines_data, training_redundant_baselines_dict),
         (testing_baselines_data, testing_redundant_baselines_dict),
         gains)
trainer.train()

training ended - session closed

In [13]:
num_downsamples = 8
batch_size = 2
lrp = 4
lr = 1.*10.**(-lrp)
network = Block_NN('network_{}_{}_{}'.format(num_downsamples,batch_size, lrp),num_downsamples, adam_inital_learning_rate=lr)
trainer = Block_Trainer(network = network, num_flatnesses = 1, downsample_keep_prob = 0.8, batch_size=batch_size, verbose=True, num_epochs = 5000)
trainer.add_data((training_baselines_data, training_redundant_baselines_dict),
         (testing_baselines_data, testing_redundant_baselines_dict),
         gains)
trainer.train()

training ended - session closed

In [14]:
num_downsamples = 9
batch_size = 2
lrp = 4
lr = 1.*10.**(-lrp)
network = Block_NN('network_{}_{}_{}'.format(num_downsamples,batch_size, lrp),num_downsamples, adam_inital_learning_rate=lr)
trainer = Block_Trainer(network = network, num_flatnesses = 1, downsample_keep_prob = 0.8, batch_size=batch_size, verbose=True, num_epochs = 5000)
trainer.add_data((training_baselines_data, training_redundant_baselines_dict),
         (testing_baselines_data, testing_redundant_baselines_dict),
         gains)
trainer.train()

training ended - session closed

In [15]:
num_downsamples = 10
batch_size = 2
lrp = 4
lr = 1.*10.**(-lrp)
network = Block_NN('network_{}_{}_{}'.format(num_downsamples,batch_size, lrp),num_downsamples, adam_inital_learning_rate=lr)
trainer = Block_Trainer(network = network, num_flatnesses = 1, downsample_keep_prob = 0.8, batch_size=batch_size, verbose=True, num_epochs = 5000)
trainer.add_data((training_baselines_data, training_redundant_baselines_dict),
         (testing_baselines_data, testing_redundant_baselines_dict),
         gains)
trainer.train()

training ended - session closed

In [16]:
num_downsamples = 2
batch_size = 2
lrp = 4
lr = 1.*10.**(-lrp)
network = Block_NN('network_{}_{}_{}_B'.format(num_downsamples,batch_size, lrp),num_downsamples, adam_inital_learning_rate=lr)
trainer = Block_Trainer(network = network, num_flatnesses = 1, downsample_keep_prob = 0.5, batch_size=batch_size, verbose=True, num_epochs = 50000)
trainer.add_data((training_baselines_data, training_redundant_baselines_dict),
         (testing_baselines_data, testing_redundant_baselines_dict),
         gains)
trainer.train()

training ended - session closed

In [ ]:
num_downsamples = 2
batch_size = 2
lrp = 4
lr = 1.*10.**(-lrp)
network = Block_NN('network_{}_{}_{}_C'.format(num_downsamples,batch_size, lrp),num_downsamples, adam_inital_learning_rate=lr)
trainer = Block_Trainer(network = network, num_flatnesses = 1, downsample_keep_prob = 1, batch_size=batch_size, verbose=True, num_epochs = 50000)
trainer.add_data((training_baselines_data, training_redundant_baselines_dict),
         (testing_baselines_data, testing_redundant_baselines_dict),
         gains)
trainer.train()

Epoch - 13174/50000 - batch:    1/  30 - (Training, Testing) -  costs: (4.6806, 5.1719) accss: (0.0667, 0.0167)

In [27]:
num_downsamples = 10
batch_size = 2
lrp = 4
lr = 1.*10.**(-lrp)
network = Block_NN_2('network_{}_{}_{}_2C'.format(num_downsamples,batch_size, lrp),num_downsamples, adam_inital_learning_rate=lr)
trainer = Block_Trainer(network = network, num_flatnesses = 100, downsample_keep_prob = 1, batch_size=batch_size, verbose=True, num_epochs = 50000)
trainer.add_data((training_baselines_data, training_redundant_baselines_dict),
         (testing_baselines_data, testing_redundant_baselines_dict),
         gains)
trainer.train()

training ended - session closed

[<tf.Tensor 'fc_layer_0/BatchNorm/Reshape_1:0' shape=(?, 2) dtype=float32>,
 <tf.Tensor 'fc_layer_1/BatchNorm/Reshape_1:0' shape=(?, 1) dtype=float32>]

In [40]:
num_downsamples = 3
batch_size = 2
lrp = 5
lr = 1.*10.**(-lrp)
network = Block_NN('network_{}_{}_{}'.format(num_downsamples,batch_size, lrp),num_downsamples, adam_inital_learning_rate=lr)
trainer = Block_Trainer(network = network, num_flatnesses = 1, downsample_keep_prob = 0.8, batch_size=batch_size, verbose=True, num_epochs = 5000)
trainer.add_data((training_baselines_data, training_redundant_baselines_dict),
         (testing_baselines_data, testing_redundant_baselines_dict),
         gains)
trainer.train()

saving trainer parameters - creating new directory - params saved1992177
training ended - session closed

In [30]:
network._layers

[]